In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import wget
import os
from datetime import datetime
from random import randrange, randint
from datetime import timedelta
import json
import re

In [2]:
# Starting from the top
print('Starting crawling script.')

Starting crawling script.


In [3]:
# Get recipe categories
base = 'https://www.myfoodandfamily.com/recipes/'
pages = {'dinner': [3, 4, 6, 8, 7, 5],
        'regional': [412, 202, 118, 486, 372, 30, 362, 140, 149, 271],
        'holiday': [597, 10, 21],
        }

In [4]:
# want category, subcategory, and link
data = []

base = 'https://www.myfoodandfamily.com/recipes/'
for category in pages:
    subcategories = pages[category]
    for subcategoryid in subcategories:
        print('Scraping category: %s on id #%i' % (category, subcategoryid))
        url = base + str(subcategoryid)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        subcategoryname = soup.find('h1').text
        cards = soup.find_all('div', class_='cls_outerCard item-card')
        for card in cards:
            link = 'https://www.myfoodandfamily.com/' + card.find('a')['href']
            image = card.find('img')['data-yo-src']
            tup = (category, subcategoryname, link, image)
            data.append(tup)

Scraping category: dinner on id #3
Scraping category: dinner on id #4
Scraping category: dinner on id #6
Scraping category: dinner on id #8
Scraping category: dinner on id #7
Scraping category: dinner on id #5
Scraping category: regional on id #412
Scraping category: regional on id #202
Scraping category: regional on id #118
Scraping category: regional on id #486
Scraping category: regional on id #372
Scraping category: regional on id #30
Scraping category: regional on id #362
Scraping category: regional on id #140
Scraping category: regional on id #149
Scraping category: regional on id #271
Scraping category: holiday on id #597
Scraping category: holiday on id #10
Scraping category: holiday on id #21


In [5]:
print('There are %i recipes' % len(data))

There are 406 recipes


In [7]:
totalData = []
for index, recipe in enumerate(data):
    if index % 20 == 0:
        print('On index %i' % index)
    category, subcategory, link, imageLink = recipe
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    title = soup.find('h1').text.strip()
    
    timeTag = soup.find('div', attrs = {'data-qa': 'qa-cook-timer'})
    if timeTag is not None:
        estimatedTime = timeTag.text.strip()
    else:
        estimatedTime = None
        
    descTag = soup.find('div', attrs = {'data-qa':'qa-prd-desc'})
    if descTag is not None:
        description = descTag.text.strip()
    else:
        description = None
    
    ingredients = []
    ingredientsList = soup.find_all('div', attrs = {'data-qa':'qa-ingreList'})
    for element in ingredientsList:
        ingredients.append(element.text.strip())
        
    procedure = []
    procedureContainer = soup.find('div', attrs = {'data-qa':'qa-recipe-make-details'})
    procedureList = procedureContainer.find_all('div', class_='krRecipeDirectionsDest')
    for step in procedureList:
        # [2:] to get rid of number
        procedure.append(step.text[2:].strip())
        
    # Don't download if already exists
    if imageLink is not None:
        downloadLocation = f'./src/recipeImages/{title}.jpg'
        if not os.path.exists(downloadLocation):
            if imageLink.startswith('//'):
                imageLink = 'https:' + imageLink
                myfile = requests.get(imageLink)
                open(downloadLocation, 'wb').write(myfile.content)
            else:
                wget.download(imageLink, downloadLocation)
        imageName = title + '.jpg'
    else:
        imageName = None
    
    recipeObject = {
        'category': category,
        'subcategory': subcategory,
        'title': title,
        'description': description,
        'ingredients': ingredients,
        'procedure': procedure,
        'imageName': imageName,
        'estimatedTime': estimatedTime
    }
    totalData.append(recipeObject)
        

On index 0
On index 20
On index 40
On index 60
On index 80
On index 100


KeyboardInterrupt: 

In [8]:
df = pd.DataFrame(totalData)

In [9]:
# Create random list of datetimes

In [10]:
df.head()

,category,description,estimatedTime,imageName,ingredients,procedure,subcategory,title
0,dinner,"Discover Fluffy Tapioca Cream, infused with eg...",None,Fluffy Tapioca Cream.jpg,"[3 Tbsp. MINUTE Tapioca, 2 cups milk, 1 egg, s...","[Combine tapioca, milk, egg yolk and 3 Tbsp. s...",Desserts Recipes,Fluffy Tapioca Cream
1,dinner,Our Fudgy Walnut Pie is super chocolatey and r...,1 Hr(s) 45 Min(s),Fudgy Walnut Pie.jpg,[1/2 pkg. (15 oz.) ready-to-use refrigerated p...,[Position oven rack in lower third of oven. Pr...,Desserts Recipes,Fudgy Walnut Pie
2,dinner,As if banana pudding weren't luscious enough o...,3 Hr(s) 45 Min(s),Deep-Dish Layered Banana Pudding.jpg,"[46 vanilla wafers, divided, 1/2 cup chopped P...","[Heat oven to 350ºF., Crush 30 wafers; mix wit...",Desserts Recipes,Deep-Dish Layered Banana Pudding
3,dinner,Make your day more colorful with our Berry Fru...,1 Hr(s) 15 Min(s),Berry Fruit Tart.jpg,[1 frozen puff pastry sheet (1/2 of 17-1/4-oz....,"[Heat oven to 400°F., Unfold pastry on lightly...",Desserts Recipes,Berry Fruit Tart
4,dinner,Start spreading the news for Pineapple-Topped ...,6 Hr(s) 20 Min(s),Pineapple-Topped New York Cheesecake.jpg,"[7 graham crackers, finely crushed (about 1 cu...","[Heat oven to 325ºF., Mix graham crumbs, butte...",Desserts Recipes,Pineapple-Topped New York Cheesecake


In [11]:
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [12]:
start = datetime.strptime('1/1/2018 12:00 AM', '%m/%d/%Y %I:%M %p')
end = datetime.strptime('11/1/2019 12:00 AM', '%m/%d/%Y %I:%M %p')

In [13]:
times = []
for i in range(len(df)):
    dates = []
    count = randint(1, 5)
    if count < 3:
        count = 1
    else:
        count -= 1
    for i in range(count):
        randomDate = random_date(start, end)
        randomDate = randomDate.strftime("%m/%d/%Y, %H:%M:%S")
        dates.append(randomDate)
    dates.sort()
    times.append(dates)

In [14]:
df['times'] = times

In [15]:
# Do favorites
isFavorite = [True if randint(1,4) == 4 else False for i in range(len(df))]
df['isFavorite'] = isFavorite

In [16]:
# Convert estimatedTime to minutes
convertedTimes = []
times = df['estimatedTime'].values
for time in times:
    if time is None:
        convertedTimes.append(None)
    else:
        numbers = re.findall(r'[0-9]+', time) 
        if len(numbers) == 2:
            total = int(numbers[1]) + int(numbers[0]) * 60
        elif len(numbers) == 1:
            total = int(numbers[0])
        else:
            print('error')
        convertedTimes.append(total)

In [17]:
df['estimatedTime'] = convertedTimes

In [54]:
def cleanList(list):
    string = ''
    for item in list:
        string = string + item + '|'
    return string.replace("'", "")[0:-1]

In [55]:
# remove [] and '' from ingredients and procedures and times
for index, row in df.iterrows():
    ingredients = row['ingredients']
    procedure = row['procedure']
    times = row['times']
    
    ingredients = cleanList(ingredients)
    procedure = cleanList(procedure)
    times = cleanList(times)
    
    df.at[index, 'ingredients'] = ingredients
    df.at[index, 'procedure'] = procedure
    df.at[index, 'times'] = times

In [57]:
# Save to csv
df.to_csv('data.csv', index = False)

In [58]:
# Change to json
jsonData = df.to_json(orient = 'records')

In [59]:
with open('./src/jsonData.json', 'w') as json_file:
    json.dump(jsonData, json_file)